In [163]:
import os
import copy
import random
import math

In [177]:
class City:
    
    def __init__(self, city):
        self.number = city[0]
        self.x = city[1]
        self.y = city[2]
        
    def __repr__(self):
        return f'({self.x},{self.y})'
        
    def __str__(self):
        return f'City No:{self.number}, X:{self.x}, Y:{self.y}'


class TSP:
    def __init__(self, path) -> None:
        self.cities = []
        self.init_tsp(path)
        
    def init_tsp(self, path):
        file = open(f'tsp_files/vlsi/{path}/{path}.tsp').readlines()
        
        self.name = file[0]
        self.size = int(file[5].split(" ")[-1].replace('\n', ''))
        
        for city_line in file[8:-1]:
            city = city_line.split(" ")
            city[1] = int(city[1])
            city[2] = int(city[2].replace('\n', ''))
            
            self.cities.append(City(city))
            
        self.cities.append(self.cities[0])
        
    def set_cities(self, cities):
        self.cities = cities
        
    def get_cities(self):
        return copy.deepcopy(self.cities)

In [267]:
class Chromosome:
    
    def __init__(self, cities):
        
        self.cities = cities
        self.fitness = None
        
    def __repr__(self):
        return f"(Fitness: {self.fitness}, {self.cities[:2]} ... {self.cities[-2:]})"
    
    def __str__(self):
        return f"Fitness: {self.fitness}\n{self.cities}"
    
    def set_fitness(self, fitness):
        self.fitness = fitness

class GeneticAlgorithm:
    
    def __init__(self, pop_size, tsp):
        self.POP_SIZE = pop_size
        self.population = []
        
        self.init_population(tsp)
        
    def init_population(self, tsp):
        
        for i in range(self.POP_SIZE):
            temp_tsp = copy.deepcopy(tsp)
            temp_city_list = temp_tsp.get_cities()
            start_city = temp_city_list[0]
            temp_city_list = temp_city_list[1:-1]
            
            random.shuffle(temp_city_list)
            temp_city_list.insert(0, start_city)
            temp_city_list.insert(len(temp_city_list), start_city)
            
#             temp_tsp.set_cities(temp_city_list)

            chrm = Chromosome(temp_city_list)
            
            self.population.append(chrm)
            
    def eval_fitness(self, indv):
        
        total_fitness = 0
        
        for i in range(len(indv.cities)-1):
            total_fitness += math.dist([indv.cities[i].x, indv.cities[i].y], [indv.cities[i+1].x, indv.cities[i+1].y])
            
        return round(total_fitness)
            
    def fit(self):
        
        for indv in self.population:
            indv.set_fitness(self.eval_fitness(indv))
            
        self.population.sort(key= lambda x: x.fitness)
        
        print(self.population)

In [268]:
path = "xqf131"
t1 = TSP(path)

ga = GeneticAlgorithm(100, t1)

In [269]:
ga.fit()

[(Fitness: 4031, [(0,13), (15,43)] ... [(41,34), (0,13)]), (Fitness: 4101, [(0,13), (10,10)] ... [(34,5), (0,13)]), (Fitness: 4157, [(0,13), (41,35)] ... [(18,29), (0,13)]), (Fitness: 4177, [(0,13), (28,16)] ... [(15,13), (0,13)]), (Fitness: 4290, [(0,13), (77,21)] ... [(71,11), (0,13)]), (Fitness: 4290, [(0,13), (18,31)] ... [(34,29), (0,13)]), (Fitness: 4320, [(0,13), (78,32)] ... [(28,16), (0,13)]), (Fitness: 4356, [(0,13), (15,25)] ... [(78,32), (0,13)]), (Fitness: 4362, [(0,13), (41,32)] ... [(15,43), (0,13)]), (Fitness: 4370, [(0,13), (71,13)] ... [(33,15), (0,13)]), (Fitness: 4389, [(0,13), (78,39)] ... [(74,12), (0,13)]), (Fitness: 4392, [(0,13), (28,40)] ... [(18,44), (0,13)]), (Fitness: 4396, [(0,13), (18,31)] ... [(18,37), (0,13)]), (Fitness: 4403, [(0,13), (25,11)] ... [(64,22), (0,13)]), (Fitness: 4404, [(0,13), (18,39)] ... [(10,10), (0,13)]), (Fitness: 4407, [(0,13), (15,13)] ... [(15,31), (0,13)]), (Fitness: 4414, [(0,13), (25,11)] ... [(15,8), (0,13)]), (Fitness: 4426,